<h1 style="text-align: center;">Assignment 3</h1>

In [1]:
import numpy as np

In [2]:
data = []
with open('imdb_labelled.txt') as f:
    data = f.readlines()

<p>Split the dataset into train/dev/test </p>

In [3]:
train_data = data[:720]

dev_data = data[720:900]

test_data = data[900:]

In [4]:
class NBC():
  def __init__(self):
    self.model_dict = {}
    self.pos_docs = 0
    self.neg_docs = 0
    self.tot_docs = 0

  def train(self, train_data):
    for data in train_data:
      label = data[len(data)-2]
      positive = True
      data = data[:len(data)-3]
      words = data.split()
      for i in range(len(words)):
        words[i] = words[i].replace(".","").replace(",","").lower()
      words = set(words)


      if label == '1':
        self.pos_docs += 1
      elif label == '0':
        self.neg_docs += 1
        positive = False
      self.tot_docs += 1

      for word in words:
        if word not in self.model_dict:
          if positive:
            self.model_dict[word] = [1, 1, 0]
          else:
            self.model_dict[word] = [1, 0, 1]
        else:
          self.model_dict[word][0] += 1
          if positive:
            self.model_dict[word][1] += 1
          else:
            self.model_dict[word][2] += 1

  def word_probability(self, word):
    return model.model_dict[word][0]/model.tot_docs

  def word_given_positive(self, word, smoothing=False):
    if smoothing:
      return (model.model_dict[word][1] + 1)/(model.pos_docs + 2)
    else:
      return model.model_dict[word][1]/model.pos_docs

  def word_given_negative(self, word, smoothing=False):
    if smoothing:
      return (model.model_dict[word][2] + 1)/(model.neg_docs + 2)
    else:
      return model.model_dict[word][2]/model.neg_docs

  def positive(self, word, smoothing=False):
    return ((self.word_given_positive(word, smoothing=smoothing))*(self.word_probability(word)))/(model.pos_docs/model.tot_docs)

  def negative(self, word, smoothing=False):
    return ((self.word_given_negative(word, smoothing=smoothing))*(self.word_probability(word)))/(model.neg_docs/model.tot_docs)

  def forward(self, test_data, smoothing=False):
    words = test_data.split()
    for i in range(len(words)):
        words[i] = words[i].replace(".","").replace(",","").lower()
    words = set(words)
    pos = model.pos_docs/model.tot_docs
    neg = model.neg_docs/model.tot_docs
    for word in words:
      if word in self.model_dict:
        pos = pos*self.word_given_positive(word, smoothing)
        neg = neg*self.word_given_negative(word, smoothing)
      else:
        if smoothing:
          pos = pos*(1/(model.pos_docs + 2))
          neg = neg*(1/(model.neg_docs + 2))
        else:
          pos = 0
          neg = 0

    return pos, neg

#Return True for positive sentiment, False for negative sentiment
  def predict(self, test_data, smoothing=False):
    pos, neg = self.forward(test_data, smoothing=smoothing)
    if pos > neg:
      return True
    else:
      return False

  def test(self, test_data, labels, smoothing=False):
    correct = 0
    total = 0
    for i in range(len(test_data)):
      if labels[i] == '1':
        label = True
      else:
        label = False
      prediction = self.predict(test_data[i], smoothing=smoothing)
      if label == prediction:
        correct += 1
      total += 1

    return correct/total
          

In [5]:
model = NBC()
model.train(train_data)

In [8]:
#P('the')
prob = model.word_probability('the')
print(prob)

0.49166666666666664


In [9]:
#P('the'|Positive)
prob = model.word_given_positive('the')
print(prob)

0.47112462006079026


In [23]:
#P('the'|Negative)
prob = model.word_given_negative('the')
print(prob)

0.5173333333333333


In [10]:
test_data[0]

"Otherwise, don't even waste your time on this.  \t0\n"

In [11]:
model.predict(test_data[0], smoothing=False)

False

In [12]:
test_labels = []
for x in test_data:
  test_labels.append(x[len(x)-2])

In [13]:
model.test(test_data, test_labels, smoothing=False)

0.39

<p>Perform 5-fold cross validation</p>

In [14]:
def validate(train_data, dev_data, smoothing=False):
  model = NBC()
  model.train(train_data)
  labels = []
  for x in dev_data:
    labels.append(x[len(x)-2])
    
  return model.test(dev_data, labels, smoothing=smoothing)

In [15]:
def FiveFoldCrossValidation(data, smoothing=False):
  accuracy = []
  train_data = data[180:900]
  dev_data = data[:180]

  accuracy.append(validate(train_data, dev_data, smoothing=smoothing))

  train_data = data[:180] + data[360:900]
  dev_data = data[180:360]

  accuracy.append(validate(train_data, dev_data, smoothing=smoothing))

  train_data = data[:360] + data[540:900]
  dev_data = data[360:540]

  accuracy.append(validate(train_data, dev_data, smoothing=smoothing))

  train_data = data[:540] + data[720:900]
  dev_data = data[540:720]

  accuracy.append(validate(train_data, dev_data, smoothing=smoothing))

  train_data = data[:720]
  dev_data = data[720:900]

  accuracy.append(validate(train_data, dev_data, smoothing=smoothing))

  return accuracy

In [16]:
accuracy = FiveFoldCrossValidation(data)
print(accuracy)

[0.6666666666666666, 0.65, 0.6277777777777778, 0.5777777777777777, 0.45555555555555555]


<p>Now we evaluate the effects of smoothing with 5-fold cross validation</p>

In [17]:
smoothing_accuracy = FiveFoldCrossValidation(data, smoothing=True)
print(smoothing_accuracy)

[0.8666666666666667, 0.8611111111111112, 0.8666666666666667, 0.9, 0.7277777777777777]


<p>From the experiments with cross validation we can determine that the best model is using the fourth train/dev split. Using this model we can derive the top ten words that predict positive and likewise the top ten words that predict negative. </p>

In [18]:
train_data = data[:540] + data[720:900]
dev_data = data[540:720]

model = NBC()
model.train(train_data)

pos = []
neg = []
for word in model.model_dict:
  pos.append((model.positive(word, smoothing=True), word))
  neg.append((model.negative(word, smoothing=True), word))


In [19]:
def sort_help(x):
  return x[0]

pos.sort(reverse=True, key=sort_help)
neg.sort(reverse=True, key=sort_help)

<p>Top ten words that predict positive and their probabilities P(Positive|word) </p>

In [24]:
print(pos[:10])

[(0.5334168650545044, 'the'), (0.2501942112517228, 'a'), (0.24207492795389043, 'and'), (0.1912709351376185, 'of'), (0.18322683038884016, 'is'), (0.15394896211836445, 'this'), (0.115774965543165, 'i'), (0.10162469197677818, 'it'), (0.09197677818151442, 'to'), (0.08896128304723717, 'in')]


<p>Top ten words that predict negative and their probabilities P(Negative|word) </p>

In [25]:
print(neg[:10])

[(0.506206896551724, 'the'), (0.17875862068965517, 'a'), (0.1724491600353669, 'and'), (0.16499381078691425, 'of'), (0.14500442086648982, 'is'), (0.12767462422634834, 'this'), (0.09687002652519892, 'i'), (0.09276038903625108, 'it'), (0.06930503978779841, 'in'), (0.06878160919540229, 'to')]


<p>It can be seen from the results that the same set of words are the top predictors for both positive and negative sentiment. This is because these are very common words in the English language. These words are included in a set of words called "Stop Words" and these words are typically filtered out of the dataset. </p>

<p>Finally we test our best model on the last 10% of the dataset that was withheld for testing. </p>

In [22]:
model.test(test_data, test_labels, smoothing=True)

0.71